# Deep Co-segmentation Experiments

## Imports

In [1]:
from dataset import iCosegDataset, PASCALVOCCosegDataset
from model import SiameseSegNet
import numpy as np
import os
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm_notebook

## Constants

In [2]:
## Debug

DEBUG = False


## Dataset
BATCH_SIZE = 2 * 1 # two images at a time for Siamese net
INPUT_CHANNELS = 3 # RGB
OUTPUT_CHANNELS = 2 # BG + FG channel

## Inference
CUDA = "0"

## Output Dir
OUTPUT_DIR = "./experiments"

os.system(f"rm -r {OUTPUT_DIR}")
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Metrics

In [3]:
def metrics(pmapA, pmapB, masksA, masksB):
    intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = 0, 0, 0, 0, 0, 0
    
    for idx in range(BATCH_SIZE//2):
        pred_maskA = torch.argmax(pmapA[idx], dim=0).cpu().numpy()
        pred_maskB = torch.argmax(pmapB[idx], dim=0).cpu().numpy()

        masksA_cpu = masksA[idx].cpu().numpy()
        masksB_cpu = masksB[idx].cpu().numpy()

        intersection_a += np.sum(pred_maskA & masksA_cpu)
        intersection_b += np.sum(pred_maskB & masksB_cpu)

        union_a += np.sum(pred_maskA | masksA_cpu)
        union_b += np.sum(pred_maskB | masksB_cpu)

        precision_a += np.sum(pred_maskA == masksA_cpu)
        precision_b += np.sum(pred_maskB == masksB_cpu)

    return intersection_a, intersection_b, union_a, union_b, precision_a, precision_b

# Experiments

## Load Deep Object Co-segmentation model trained on Pascal VOC 

In [4]:
LOAD_CHECKPOINT = "/home/SharedData/intern_sayan/PASCAL_coseg/"
root_dir = "/home/SharedData/intern_sayan/iCoseg/"

image_dir = os.path.join(root_dir, "images")
mask_dir = os.path.join(root_dir, "ground_truth")

iCoseg_dataset = iCosegDataset(image_dir=image_dir,
                               mask_dir=mask_dir)
dataloader = DataLoader(iCoseg_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

model = SiameseSegNet(input_channels=INPUT_CHANNELS,
                          output_channels=OUTPUT_CHANNELS,
                          gpu=CUDA)
if DEBUG:
    print(model)

FloatTensor = torch.FloatTensor
LongTensor = torch.LongTensor

if CUDA is not None:
    os.environ["CUDA_VISIBLE_DEVICES"] = CUDA

    model = model.cuda()

    FloatTensor = torch.cuda.FloatTensor
    LongTensor = torch.cuda.LongTensor

if LOAD_CHECKPOINT:
    model.load_state_dict(torch.load(os.path.join(LOAD_CHECKPOINT, "coseg_model_best.pth")))

## VOC + iCoseg [Car]  

iCoseg class indices = {5}

In [5]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([5,5]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"car_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"car_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"car_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [4.050945997238159 secs]
Precision : [0.013725797706675306]
IoU : [0.5239575005999502]


## VOC + iCoseg [People]
iCoseg class indices = {1,4,26,27,28}

In [6]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([1,1]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([4,4]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([26,26]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([26,27]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([27,27]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([27,28]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([28,28]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"people_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"people_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"people_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [25.077014923095703 secs]
Precision : [0.16889885875666252]
IoU : [0.3234308910308019]


## VOC + iCoseg [Goose]
iCoseg class indices = {10}

In [7]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([10,10]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"goose_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"goose_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"goose_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [7.623862266540527 secs]
Precision : [0.038232476540443684]
IoU : [0.3848175873343659]


## VOC + iCoseg [Airplane]  

iCoseg class indices = {12,13,14}

In [8]:
def infer():
    model.eval()

    intersection, union, precision = 0, 0, 0
    t_start = time.time()

    for batch_idx, batch in tqdm_notebook(enumerate(dataloader)):
        images = batch["image"].type(FloatTensor)
        labels = batch["label"].type(LongTensor)
        masks  = batch["mask"].type(FloatTensor)
        
        if torch.equal(batch["label"], torch.from_numpy(np.array([12,12]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([12,13]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([13,13]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([13,14]))) or \
           torch.equal(batch["label"], torch.from_numpy(np.array([14,14]))):

            # pdb.set_trace()

            pairwise_images = [(images[2*idx], images[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_labels = [(labels[2*idx], labels[2*idx+1]) for idx in range(BATCH_SIZE//2)]
            pairwise_masks  = [(masks[2*idx], masks[2*idx+1]) for idx in range(BATCH_SIZE//2)]

            # pdb.set_trace()

            imagesA, imagesB = zip(*pairwise_images)
            labelsA, labelsB = zip(*pairwise_labels)
            masksA, masksB = zip(*pairwise_masks)

            # pdb.set_trace()

            imagesA, imagesB = torch.stack(imagesA), torch.stack(imagesB)
            labelsA, labelsB = torch.stack(labelsA), torch.stack(labelsB)
            masksA, masksB = torch.stack(masksA).long(), torch.stack(masksB).long()

            # pdb.set_trace()

            eq_labels = []

            for idx in range(BATCH_SIZE//2):
                if torch.equal(labelsA[idx], labelsB[idx]):
                    eq_labels.append(torch.ones(1).type(LongTensor))
                else:
                    eq_labels.append(torch.zeros(1).type(LongTensor))

            eq_labels = torch.stack(eq_labels)

            # pdb.set_trace()

            masksA = masksA * eq_labels.unsqueeze(1)
            masksB = masksB * eq_labels.unsqueeze(1)

            imagesA_v = torch.autograd.Variable(FloatTensor(imagesA))
            imagesB_v = torch.autograd.Variable(FloatTensor(imagesB))

            pmapA, pmapB, similarity = model(imagesA_v, imagesB_v)

            # pdb.set_trace()

            res_images, res_masks, gt_masks = [], [], []

            for idx in range(BATCH_SIZE//2):
                res_images.append(imagesA[idx])
                res_images.append(imagesB[idx])

                res_masks.append(torch.argmax((pmapA * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))
                res_masks.append(torch.argmax((pmapB * similarity.unsqueeze(2).unsqueeze(2))[idx],
                                              dim=0).reshape(1, 512, 512))

                gt_masks.append(masksA[idx].reshape(1, 512, 512))
                gt_masks.append(masksB[idx].reshape(1, 512, 512))

            # pdb.set_trace()

            images_T = torch.stack(res_images)
            masks_T = torch.stack(res_masks)
            gt_masks_T = torch.stack(gt_masks)


            # metrics - IoU & precision
            intersection_a, intersection_b, union_a, union_b, precision_a, precision_b = metrics(pmapA,
                                                                                                 pmapB,
                                                                                                 masksA,
                                                                                                 masksB)

            intersection += intersection_a + intersection_b
            union += union_a + union_b

            precision += (precision_a / (512 * 512)) + (precision_b / (512 * 512))


            # pdb.set_trace()

            torchvision.utils.save_image(images_T,
                                         os.path.join(OUTPUT_DIR,f"airplane_{batch_idx}_images.png"),
                                         nrow=2)
            torchvision.utils.save_image(masks_T,
                                         os.path.join(OUTPUT_DIR, f"airplane_{batch_idx}_masks.png"),
                                         nrow=2)
            torchvision.utils.save_image(gt_masks_T,
                                         os.path.join(OUTPUT_DIR, f"airplane_{batch_idx}_gt_masks.png"),
                                         nrow=2)

    delta = time.time() - t_start

    print(f"\nTime elapsed: [{delta} secs]\nPrecision : [{precision/(len(dataloader) * BATCH_SIZE)}]\nIoU : [{intersection/union}]")
    
    
infer()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))



Time elapsed: [15.512147426605225 secs]
Precision : [0.10875255667903341]
IoU : [0.30711214210225235]
